In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import PIL
from PIL import Image

import cv2

import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import transforms

import torch.utils.data
from torch.utils.data import Dataset, DataLoader
import random

import albumentations
from albumentations.pytorch.transforms import ToTensorV2

In [2]:
def get_instance_objectdetection_model(num_classes,path_weight):
    # load an instance segmentation model pre-trained on COCO
    create_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False,pretrained_backbone=False)

    # get the number of input features for the classifier
    in_features = create_model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    create_model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    create_model.load_state_dict(torch.load(path_weight,map_location=torch.device('cpu')))

    return create_model

In [3]:
#os.listdir("/kaggle/input/gwd-thirditeration-weights/customtrained_fasterrcnn_resnet50_fpn_augementation_28052020_1340.pth")

In [4]:
#path_weight = "/kaggle/input/global_wheat_detection/fasterrcnn_resnet50_fpn_best.pth"
#path_weight = "/kaggle/input/gwd-augmentation-training-weights-ver01/customtrained_fasterrcnn_resnet50_fpn_augementation (1).pth"
path_weight = "/kaggle/input/gwd-thirditeration-weights/customtrained_fasterrcnn_resnet50_fpn_augementation_28052020_1340.pth"

In [5]:
num_classes = 2
# Why 2 classes - background and wheat-heads
trained_model = get_instance_objectdetection_model(num_classes,path_weight)

In [6]:
def get_test_transform():
    return albumentations.Compose([
        # A.Resize(512, 512),
        ToTensorV2(p=1.0)
    ])

In [7]:
class GlobalWheatDetectionTestDataset(torch.utils.data.Dataset):
    # first lets start with __init__ and initialize any objects
    def __init__(self,input_df,input_dir,transforms=None):
        
        self.df=input_df
        
        self.list_images = list(self.df['image_id'].unique())
        
        self.image_dir=input_dir
        
        self.transforms = transforms
    
    # next lets define __getitem__
    # very important to note what it returns for EACH image:
    # I. image - a PIL image of size (H,W) for ResNet50 FPN image should be scaled
    
    # II. image_id 
    
    def __getitem__(self,idx):
        
        # II. image_id
        img_id = self.list_images[idx]
        # I. Input image
        # Specifications: A.RGB format B. scaled (0,1) C. size (H,W) D. PIL format
        
        img = cv2.imread(self.image_dir+"/"+img_id+".jpg")
        img_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img_scaled = img_RGB/255.0
        img_final = img_scaled
        ret_image = {}
        ret_image['image']=img_final
        if self.transforms is not None:
            res = self.transforms(**ret_image)
            img_final = res['image']
            #img_final = torch.tensor(img_final, dtype=torch.float32)
        
        return img_final, img_id
    
    # next lets define __len__    
    def __len__(self):
        
        return len(self.df['image_id'].unique())

In [8]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [9]:
df_test=pd.read_csv("/kaggle/input/global-wheat-detection/sample_submission.csv")

In [10]:
test_dir="/kaggle/input/global-wheat-detection/test/"

In [11]:
df_test.head()

,image_id,PredictionString
0,aac893a91,1.0 0 0 50 50
1,51f1be19e,1.0 0 0 50 50
2,f5a1f0358,1.0 0 0 50 50
3,796707dd7,1.0 0 0 50 50
4,51b3e36ab,1.0 0 0 50 50


In [12]:
test_dataset = GlobalWheatDetectionTestDataset(df_test,test_dir,get_test_transform())

In [13]:
test_dataloader = DataLoader(test_dataset, batch_size=8,shuffle=False, num_workers=1,collate_fn=collate_fn)

In [14]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [15]:
detection_threshold = 0.45

In [16]:
def format_prediction_string(boxes, scores): ## Define the formate for storing prediction results
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)

In [17]:
images, img_ids = next(iter(test_dataloader))

In [18]:
## Lets make the prediction
results=[]
trained_model.to(device)
trained_model.eval()
images = []
outputs =[]
for images_, image_ids in test_dataloader:    

    images = list(image.to(device,dtype=torch.float) for image in images_)
    outputs = trained_model(images)

    for i, image in enumerate(images):

        boxes = outputs[i]['boxes'].data.cpu().numpy()    ##Formate of the output's box is [Xmin,Ymin,Xmax,Ymax]
        scores = outputs[i]['scores'].data.cpu().numpy()
        
        boxes = boxes[scores >= detection_threshold].astype(np.int32) #Compare the score of output with the threshold and
        scores = scores[scores >= detection_threshold]                    #slelect only those boxes whose score is greater
                                                                          # than threshold value
        image_id = image_ids[i]
        
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]         
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]         #Convert the box formate to [Xmin,Ymin,W,H]
        
        
            
        result = {                                     #Store the image id and boxes and scores in result dict.
            'image_id': image_id,
            'PredictionString': format_prediction_string(boxes, scores)
        }

        
        results.append(result)              #Append the result dict to Results list

test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.head()

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/opt/conda/conda-bld/pytorch_1587428398394/work/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


,image_id,PredictionString
0,aac893a91,0.9980 557 533 129 184 0.9946 68 0 100 159 0.9...
1,51f1be19e,0.9979 606 79 162 177 0.9959 839 261 136 204 0...
2,f5a1f0358,0.9968 136 747 166 123 0.9967 938 426 85 192 0...
3,796707dd7,0.9947 375 630 103 118 0.9945 940 67 83 104 0....
4,51b3e36ab,0.9986 833 448 187 147 0.9982 541 27 250 132 0...


In [19]:
test_df.to_csv('submission.csv', index=False)